# Необходимые библиотеки

In [1]:
import pandas as pd
import numpy as np

In [2]:
from tqdm import tqdm

tqdm.pandas()

In [3]:
import os

# Собираем stations.csv

In [4]:
path_to_extract_from = os.path.join('_data', 'stations')

objects = os.listdir(path_to_extract_from)

In [5]:
'''
в некоторых файлах были признаки, не повторяющиеся ни в одной другой таблице ('Unnamed: 7', 'landmark', 'dateCreated').
от них просто избавился. признак города встречается только в более поздних датасетах, вероятно это связано с расширением
территории, на которой работала компания, поэтому по умолчаню поставил город Chicago
'''

stations = pd.DataFrame(columns=['id', 'name', 'city', 'latitude', 'longitude', 'dpcapacity', 'online_date'])

for obj_name in objects:
    if obj_name.endswith('.csv'):
        obj = pd.read_csv(os.path.join(path_to_extract_from, obj_name))

    else:
        obj = pd.read_excel(os.path.join(path_to_extract_from, obj_name))

    if not 'city' in obj.columns:
            obj['city'] = 'Chicago'

    if 'online date' not in obj.columns and 'online_date' not in obj.columns:
         obj['online_date'] = pd.NA

    obj = obj.drop(['Unnamed: 7', 'landmark', 'dateCreated'], axis=1, errors='ignore')

    stations = pd.concat([stations, obj])

C:\Users\User\AppData\Local\Temp\ipykernel_11496\787698988.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  stations = pd.concat([stations, obj])


In [6]:
stations['online_date'] = stations['online_date'].fillna(stations['online date'])
stations = stations.drop('online date', axis=1)

In [7]:
stations.to_csv('stations.csv', index=False)

# Собираем датасет поездок

In [8]:
path_to_extract_from = os.path.join('_data', 'trips')

objects = os.listdir(path_to_extract_from)

In [9]:
# названия однородных признаков в разных таблицах сильно варьируются. здесь мы приводим каждую таблицу к единому стандарту

column_mapping = {
    'ride_id': ['ride_id', 'trip_id', '01 - Rental Details Rental ID'],
    'started_at': ['started_at', 'starttime', 'start_time', '01 - Rental Details Local Start Time'],
    'ended_at': ['ended_at', 'stoptime', 'end_time', '01 - Rental Details Local End Time'],
    'bike_id': ['bikeid', '01 - Rental Details Bike ID'],
    'trip_duration': ['tripduration', '01 - Rental Details Duration In Seconds Uncapped'],
    'start_station_id': ['start_station_id', 'from_station_id', '03 - Rental Start Station ID'],
    'start_station_name': ['start_station_name', 'from_station_name', '03 - Rental Start Station Name'],
    'end_station_id': ['end_station_id', 'to_station_id', '02 - Rental End Station ID'],
    'end_station_name': ['end_station_name', 'to_station_name', '02 - Rental End Station Name'],
    'user_type': ['member_casual', 'usertype', 'User Type'],
    'gender': ['gender', 'Member Gender'],
    'birth_year': ['birthyear', '05 - Member Details Member Birthday Year'],
    'start_lat': ['start_lat'],
    'start_lng': ['start_lng'],
    'end_lat': ['end_lat'],
    'end_lng': ['end_lng'],
    'rideable_type': ['rideable_type']
}

rename_map = {old_name: new_name for new_name, old_names in column_mapping.items() for old_name in old_names}

In [10]:
all_data = []

for obj_name in tqdm(objects):
    file_path = os.path.join(path_to_extract_from, obj_name)
    df = pd.read_csv(file_path, low_memory=False)
    df.rename(columns=rename_map, inplace=True)

    available_columns = [col for col in column_mapping.keys() if col in df.columns]
    df = df[available_columns]

    df['started_at'] = pd.to_datetime(df['started_at'])
    df['ended_at'] = pd.to_datetime(df['ended_at'])

    all_data.append(df)

trips = pd.concat(all_data, ignore_index=True)

100%|██████████| 88/88 [03:32<00:00,  2.41s/it]


In [11]:
trips[trips.select_dtypes('number').columns] = trips[trips.select_dtypes('number').columns].progress_applymap(str)

  0%|          | 0/282954198 [00:00<?, ?it/s]c:\Users\User\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\std.py:917: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  return getattr(df, df_function)(wrapper, **kwargs)
100%|██████████| 282954198/282954198 [03:55<00:00, 1202633.68it/s]


In [12]:
trips['start_station_id'] = trips['start_station_id'].astype(str)
trips['end_station_id'] = trips['end_station_id'].astype(str)
trips['ride_id'] = trips['ride_id'].astype(str)
trips['trip_duration'] = trips['trip_duration'].astype(str)

In [13]:
'''
вроде ничего не потерял. единственное, очень много пропусков в trips
'''

os.makedirs('chunks', exist_ok=True)

chunk_size = 1_000_000

for i in tqdm(range(0, len(trips), chunk_size)):
    chunk = trips.iloc[i: i + chunk_size]
    chunk.to_parquet(f'chunks/chunk{i//chunk_size}.parquet', index=False)

100%|██████████| 48/48 [02:13<00:00,  2.78s/it]


# Очистка данных

In [4]:
df = pd.DataFrame()

for i in tqdm(range(len(os.listdir('chunks')))):
    chunk_file = f'chunks/chunk{i}.parquet'

    chunk = pd.read_parquet(chunk_file)
    df = pd.concat([df, chunk], ignore_index=True)

100%|██████████| 48/48 [02:43<00:00,  3.41s/it]


## Избавляемся от выбросов

In [5]:
df['trip_duration'] = df['ended_at'] - df['started_at']
df = df[(df['trip_duration'] > pd.Timedelta(minutes=0)) & (df['trip_duration'] < pd.Timedelta(days=1))]

In [6]:
Q1 = df['trip_duration'].quantile(0.25)
Q3 = df['trip_duration'].quantile(0.75)
IQR = Q3 - Q1

df = df[df['trip_duration'] < Q3 + 3 * IQR]
df = df[df['trip_duration'] > pd.Timedelta(minutes=2)]

ставя такой порог (1 час 10 секунд), я потенцаинльно теряю данные о поездках по тарифу "Day Pass". но в общей выборке поездки такой длительности почти не представлены, из-за чего отличить их от простых выбросов (некорректный сбор данных) не представляется возможным.

In [7]:
df[['start_lat', 'start_lng', 'end_lat', 'end_lng', 'birth_year']] = \
    df[['start_lat', 'start_lng', 'end_lat', 'end_lng', 'birth_year']].astype(np.float32)

In [8]:
LEFT_LOWER_BOUND = {'lng': -89.2, 'lat': 40.8}
RIGHT_UPPER_BOUND = {'lng': -86.1, 'lat': 42.9}

In [9]:
df = df[
    ((df['start_lat'] < RIGHT_UPPER_BOUND['lat']) | df['start_lat'].isna()) &
    ((df['start_lat'] > LEFT_LOWER_BOUND['lat']) | df['start_lat'].isna()) &
    ((df['start_lng'] < RIGHT_UPPER_BOUND['lng']) | df['start_lng'].isna()) &
    ((df['start_lng'] > LEFT_LOWER_BOUND['lng']) | df['start_lng'].isna()) &
    ((df['end_lat'] < RIGHT_UPPER_BOUND['lat']) | df['end_lat'].isna()) &
    ((df['end_lat'] > LEFT_LOWER_BOUND['lat']) | df['end_lat'].isna()) &
    ((df['end_lng'] < RIGHT_UPPER_BOUND['lng']) | df['end_lng'].isna()) &
    ((df['end_lng'] > LEFT_LOWER_BOUND['lng']) | df['end_lng'].isna())
]

In [10]:
df = df[((df['birth_year'] > 1950) & ((df['started_at'].dt.year - df['birth_year']) >= 10)) | (df['birth_year'].isna())]

## Избавляемся от дубликатов

In [11]:
mask_to_drop = df.duplicated(subset=['started_at', 'ended_at', 'user_type',
                                     'start_station_name', 'end_station_name', 'rideable_type'])

In [12]:
df = df[~mask_to_drop]

## Приводим к двум .parquet файлам

In [13]:
before_2020 = df[df['started_at'] < pd.Timestamp(year=2020, month=1, day=1)].copy()
after_2020 = df[df['started_at'] >= pd.Timestamp(year=2020, month=1, day=1)].copy()

In [14]:
before_2020 = before_2020.drop(columns=['start_lat', 'start_lng', 'end_lat', 'end_lng', 'rideable_type'])
after_2020 = after_2020.drop(columns=['bike_id', 'gender', 'birth_year'])

In [15]:
before_2020.to_parquet('before_2020.parquet', index=False)
after_2020.to_parquet('after_2020.parquet', index=False)

# Собираем данные о всех станциях и считаем дохода с поездки

In [16]:
before_2020 = pd.read_parquet('before_2020.parquet')
after_2020 = pd.read_parquet('after_2020.parquet')

In [17]:
tmp = after_2020.dropna(subset=['start_station_name', 'end_station_name'])

start = tmp.groupby('start_station_name', as_index=False).agg({
    'start_lat': lambda x: x.mode()[0] if not x.mode().empty else None,
    'start_lng': lambda x: x.mode()[0] if not x.mode().empty else None
})
start.columns = ['station_name', 'lat', 'lng']

end = tmp.groupby('end_station_name', as_index=False).agg({
    'end_lat': lambda x: x.mode()[0] if not x.mode().empty else None,
    'end_lng': lambda x: x.mode()[0] if not x.mode().empty else None
})
end.columns = ['station_name', 'lat', 'lng']

stations = pd.concat([start, end]).drop_duplicates('station_name').reset_index(drop=True)

In [18]:
stations_csv = pd.read_csv('stations.csv')
stations_csv = stations_csv.drop(columns=['id', 'dpcapacity', 'online_date', 'city'])
stations_csv = stations_csv.rename({'name': 'station_name', 'latitude': 'lat', 'longitude': 'lng'}, axis=1)
stations_csv = stations_csv.drop_duplicates()

In [19]:
stations = pd.concat([stations, stations_csv]).drop_duplicates('station_name').reset_index(drop=True)

In [20]:
before_2020['user_type'] = before_2020['user_type'].map({'Subscriber': 'member', 'Customer': 'casual'})
before_2020 = before_2020.dropna(subset=['user_type'])

before_2020 = before_2020.drop(columns=['bike_id', 'start_station_id', 'end_station_id', 'ride_id'])

before_2020['trip_duration'] = before_2020['trip_duration'].dt.seconds
before_2020 = before_2020.sort_values('started_at')

In [21]:
before_2020['age'] = before_2020.started_at.dt.year - before_2020['birth_year']
before_2020 = before_2020.drop(columns=['birth_year'])

before_2020['age'] = before_2020['age'].fillna(before_2020['age'].mean())

In [22]:
after_2020 = after_2020.drop(columns=['start_station_id', 'end_station_id', 'ride_id'])

after_2020['trip_duration'] = after_2020['trip_duration'].dt.seconds
after_2020 = after_2020.sort_values('started_at')

В датасете много безымянных станций, чьи координаы неоднократно повторяются. Вынесем их также

## Выносим nan-станции

In [23]:
unknown_counter = 0
station_map = {}

def start_fill_with_unique_id(x):
    global unknown_counter
    if pd.isna(x['start_station_name']):
        coords = (x['start_lat'], x['start_lng'])
        if coords in station_map:
            return station_map[coords]
        else:
            val = f"unknown_station{unknown_counter}"
            station_map[coords] = val
            unknown_counter += 1
            return val
    return x['start_station_name']

def end_fill_with_unique_id(x):
    global unknown_counter
    if pd.isna(x['end_station_name']) and not pd.isna(x['end_lat']) and not pd.isna(x['end_lng']):
        coords = (x['end_lat'], x['end_lng'])
        if coords in station_map:
            return station_map[coords]
        else:
            val = f"unknown_station{unknown_counter}"
            station_map[coords] = val
            unknown_counter += 1
            return val
    return x['end_station_name']

after_2020['start_station_name'] = after_2020.progress_apply(start_fill_with_unique_id, axis=1)
# print('готово')
after_2020['end_station_name'] = after_2020.progress_apply(end_fill_with_unique_id, axis=1)
# print('готово')
# не работает tqdm почему-то

100%|██████████| 24723321/24723321 [04:11<00:00, 98453.71it/s] 


In [24]:
uknown_stations_data = [{'station_name': name, 'lat': lat, 'lng': lng}
        for (lat, lng), name in station_map.items()]

uknown_stations = pd.DataFrame(uknown_stations_data, columns=['station_name', 'lat', 'lng'])

In [25]:
stations = pd.concat([stations, uknown_stations]).drop_duplicates('station_name').reset_index(drop=True)

## Считаем доход от каждой поездки

In [26]:
after_2020 = pd.read_parquet('after_2020_income_included.parquet')

In [27]:
after_2020 = after_2020.drop(columns=['income'])

In [28]:
prices = pd.read_csv('prices.csv')
prices.columns = prices.columns.str.lower()
prices = prices[prices['year'] >= 2020]

prices = prices[(prices['tariff_type'].isin(['Single Ride', 'Annual Pass']))]

In [29]:
prices['bike_type'] = prices['bike_type'].map({'Classic Bike': 'classic_bike',
                                               'Classic Bikes': 'classic_bike',
                                                'Ebike': 'electric_bike',
                                                'E-Bikes': 'electric_bike',
                                                'Scooters': 'electric_scooter'
                                               })

prices['tariff_type'] = prices['tariff_type'].map({'Single Ride': 'casual',
                                                    'Day Pass': 'casual',
                                                    'Annual Pass': 'member',
                                                    'Subscription': 'member',
                                                    'Member': 'member'
                                                })

In [30]:
tariff_lookup = {
    (row['year'], row['tariff_type'], row['bike_type']): (
        row['unlock_fee'],
        row['free_period'],
        row['per_minute_rate']
    )
    for _, row in prices.iterrows()
}

In [31]:
after_2020['year'] = after_2020['started_at'].dt.year

In [32]:
nulls = 0
nulls_counts = {2020: 0, 2021: 0, 2022: 0, 2023: 0, 2024: 0, 2025: 0}

def count_ride_income_fast(x):
    global nulls
    year = x['year']
    user_type = x['user_type']
    rideable_type = x['rideable_type']
    duration = x['trip_duration'] // 60 + 1

    if rideable_type == 'docked_bike':
        rideable_type = 'classic_bike'
    elif rideable_type == 'electric_scooter':
        rideable_type = 'electric_bike'

    key = (year, user_type, rideable_type)
    if key not in tariff_lookup.keys():
        nulls += 1
        nulls_counts[year] += 1

    unlock_fee, free_minutes, rate = tariff_lookup.get(
        key, (1, 45, 0.18)
    )

    income = unlock_fee + max(duration - free_minutes, 0) * rate
    return income


after_2020['income'] = after_2020.progress_apply(count_ride_income_fast, axis=1)

100%|██████████| 24720177/24720177 [05:36<00:00, 73559.14it/s] 


# Выносим станции-призраки в отдельный датасет

In [35]:
after_2020 = after_2020.reset_index(drop=True)
after_2020 = after_2020.dropna(subset=['start_station_name', 'end_station_name'])

In [36]:
THRESHHOLD_FOR_STATIONS = 1000

all_years = pd.concat([before_2020, after_2020])

ssc = all_years.start_station_name.value_counts()
ssc = ssc > THRESHHOLD_FOR_STATIONS
ssc = ssc.rename('start_to_left')

esc = all_years.end_station_name.value_counts()
esc = esc > THRESHHOLD_FOR_STATIONS
esc = esc.rename('end_to_left')

all_years = all_years.merge(ssc, on='start_station_name', how='left')
all_years = all_years.merge(esc, on='end_station_name', how='left')

In [37]:
all_yaears = all_years[(all_years['start_to_left']) & (all_years['end_to_left'])]

In [38]:
ghost_stations = esc[~esc].index
ghost_stations = stations[stations['station_name'].isin(ghost_stations)]

In [39]:
before_2020['trip_duration'] = before_2020['trip_duration'].astype(np.int32)
before_2020['age'] = before_2020['age'].astype(np.int32)

after_2020['trip_duration'] = after_2020['trip_duration'].astype(np.int32)
after_2020['income'] = after_2020['income'].astype(np.float32)

In [40]:
after_2020.to_parquet('after_2020_income_included.parquet')
before_2020.to_parquet('before_2020.parquet')

# Считаем все доходы и расходы

Мы выбрали распределение подписок как 35% на 65% опираясь [на это исследование](https://medium.com/%40aswinpushkar11/exploratory-data-analysis-of-divvy-bike-sharing-a5ce4498e723)

Число подписчиков Divvy оценить тяжелее, так как нет публичных отчетов. Оценка составлена на [заявлении Чикагской администрации](https://www.chicago.gov/city/en/depts/cdot/provdrs/bike/news/2023/april/divvy-for-the-entire-city--divvy-service-hits-all-50-wards.html?)

>Last year (2022) Divvy had nearly 550,000 unique riders and reached over 43,000 members.

In [41]:
after_2020 = pd.read_parquet('after_2020_income_included.parquet')

In [42]:
def calculate_revenues(df, business_params):
    results = []

    for year in range(2020, 2026):
        rides = df[df['started_at'].dt.year == year]
        ride_revenue = rides['income'].sum()

        subs = business_params['annual_pass'].get(year, {})
        if 'price' in subs:
            sub_revenue = subs['count'] * subs['price']
        else:
            sub_revenue = (subs['count'] * 0.35 * subs['price_basic'] +
                          subs['count'] * 0.65 * subs['price_premium'])

        total_revenue = ride_revenue + sub_revenue

        results.append({
            'year': year,
            'total_revenue': total_revenue
        })

    return pd.DataFrame(results)

In [43]:
def calculate_expenses(df, business_params):
    expenses = []

    for year in range(2020, 2026):
        year_data = df[df['started_at'].dt.year == year]
        unique_stations = pd.concat([
            year_data['start_station_name'],
            year_data['end_station_name']
        ]).nunique()

        station_costs = unique_stations * business_params['station_rent'][year] * 12

        salary_costs = (business_params['employees_count'][year] *
                       business_params['employee_salary'][year] * 12)

        total_expenses = station_costs + salary_costs

        expenses.append({
            'year': year,\
            'total_expenses': total_expenses
        })

    return pd.DataFrame(expenses)

In [44]:
df = after_2020.copy()
df['started_at'] = pd.to_datetime(df['started_at'])

price_count_ect = {
    'station_rent': {
        2020: 300,
        2021: 325,
        2022: 350,
        2023: 375,
        2024: 400,
        2025: 425
    }, # средняя стоимость аренды и обсуживания одной станции и всех ее велосипедов в месяц
    'employee_salary': {
        2020: 3000,
        2021: 3100,
        2022: 3200,
        2023: 3300,
        2024: 3400,
        2025: 3500
    }, # средняя зарплата сотрудника в месяц (средняя с 2020)
    'employees_count': {
        2020: 150,
        2021: 150,
        2022: 155,
        2023: 155,
        2024: 155,
        2025: 160
    },
    'annual_pass': {
        2020: {'count': 35000, 'price': 99},
        2021: {'count': 40000, 'price': 108},
        2022: {'count': 45000, 'price': 119},
        2023: {'count': 50000, 'price_basic': 130.9, 'price_premium': 199},
        2024: {'count': 55000, 'price_basic': 143.9, 'price_premium': 199},
        2025: {'count': 60000, 'price_basic': 143.9, 'price_premium': 199}
    }
}

In [45]:
revenues = calculate_revenues(df, price_count_ect)
expenses = calculate_expenses(df, price_count_ect)

In [46]:
final_report = revenues.merge(expenses, on='year')
final_report['profit'] = final_report['total_revenue'] - final_report['total_expenses']

final_report.to_csv('summary.csv', index=False)

## Юнит экономика для каждого вида байка

In [47]:
final_report = pd.read_csv('summary.csv')
bikes_amount = pd.read_csv('bikes_amount.tsv', sep='\t')

final_report = final_report.merge(bikes_amount, on='year')

In [48]:
final_report['profit_per_bike'] = final_report['profit'] / (final_report['classic_bikes'] + final_report['electric_bikes'])

In [49]:
after_2020_temp = after_2020.copy()
after_2020_temp['year'] = after_2020_temp['started_at'].dt.year

after_2020_temp['rideable_type'] = after_2020_temp['rideable_type'].map({'classic_bike': 'classic_bike',
                                               'docked_bike': 'classic_bike',
                                                'electric_scooter': 'electric_bike',
                                                'electric_bike': 'electric_bike',
                                                'Scooters': 'electric_bike'
                                               })

In [50]:
profits = pd.DataFrame(columns=['year', 'bike_type', 'profit'])

profits = profits.astype({
    'year': 'int16',
    'bike_type': 'object',
    'profit': 'int64'
})

for year in range(2020, 2025):
    for bike_type in ['electric_bike', 'classic_bike']:
        p_type = after_2020_temp[after_2020_temp['year'] == year]['rideable_type'].value_counts(normalize=True).get(bike_type, 0)

        P = final_report[final_report['year'] == year]['profit_per_bike'].values[0]
        bike_share = final_report[final_report['year'] == year][bike_type + 's'] / (final_report['classic_bikes'] + 
                                                                              final_report['electric_bikes'])
        bike_share = bike_share.dropna().values[0]
        
        profit = round(p_type * P / bike_share, 2)
        profits.loc[len(profits)] = [year, bike_type, profit]

In [51]:
final_report.to_csv('final_report.csv', index=False)
profits.to_csv('profits.csv', index=False)